In [5]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv('Files/Day4.txt', header=None)

In [201]:
#  These is probably a better way of doing this, but I am not sure how.
keys = data.iloc[0,:].astype(int).tolist()
bingo_df = pd.DataFrame(data.iloc[1:,0])
bingo_card_dimension = 5
bingo_card_deck = []

for i in range(0, bingo_df.shape[0], bingo_card_dimension):
    bingo_card = pd.DataFrame(bingo_df.iloc[i:i+bingo_card_dimension,:])
    bingo_card = bingo_card.apply(lambda line: line.str.strip(). # This is to remove leading and trailing white space
                                  str.replace('  ', ' '). # For single digits in the middle, this makes it one space
                                  str.split(' ')) # Now that everything is one space, we can split it easily
    bingo_card_deck.append(
        {
            "Deck" : pd.DataFrame(list(bingo_card[0])),
            "Selected" : pd.DataFrame(0, index=range(bingo_card_dimension), columns=range(bingo_card_dimension)),
            "Won" : False
        }
    )
    
bingo_card_deck[0]

{'Deck':     0   1   2   3   4
 0  50  83   3  31  16
 1  47   9  94  10  86
 2  61  22  53  46  74
 3  77  41  79  55  62
 4  97  78  43  73  40, 'Selected':    0  1  2  3  4
 0  0  0  0  0  0
 1  0  0  0  0  0
 2  0  0  0  0  0
 3  0  0  0  0  0
 4  0  0  0  0  0, 'Won': False}

In [183]:
winner = False
for pulled_number in keys:
    for indiv_bingo_card in bingo_card_deck:
        if str(pulled_number) in indiv_bingo_card['Deck'].values:
            where_am_i = indiv_bingo_card['Deck'].isin([str(pulled_number)])
            coord = np.where(where_am_i)
            coordinates = [(x,y) for x, y in zip(coord[0], coord[1])]
            
            for coordinate in coordinates: # I dont think this is needed, but I am going to keep it just in case.
                indiv_bingo_card['Selected'].iloc[coordinate] = 1
            
            # Do any of the rows or columns add up to 5?
            # What about the two diagonals?
            winner = any([any(indiv_bingo_card['Selected'].apply(sum, axis = 0) == bingo_card_dimension), 
                          any(indiv_bingo_card['Selected'].apply(sum, axis = 1) == bingo_card_dimension),
                          np.diag(indiv_bingo_card['Selected']).sum() == bingo_card_dimension,
                          np.diag(indiv_bingo_card['Selected'].iloc[::-1]).sum() == bingo_card_dimension])
            
            if winner:
                print('Winner!')
                winning_card = indiv_bingo_card
        if winner: break
    if winner: break

Winner!


In [184]:
winning_card

{'Deck':     0   1   2   3   4
 0  84  78   3  44  96
 1  59  86  70  80  48
 2  93  88  52  43  61
 3  95  66  46  62  58
 4   5  25   6  85  99, 'Selected':    0  1  2  3  4
 0  0  0  0  0  0
 1  0  0  0  0  1
 2  0  1  0  1  0
 3  1  1  1  1  1
 4  0  0  0  0  0}

In [188]:
not_selectd_values = winning_card['Selected'].isin([str(0)])
coord = np.where(not_selectd_values)
coordinates = [(x,y) for x, y in zip(coord[0], coord[1])]
sum_of_not_selected_numbers = 0
for coordinate in coordinates:
    sum_of_not_selected_numbers += int(winning_card['Deck'].iloc[coordinate])
sum_of_not_selected_numbers * pulled_number

67716

In [202]:
# Part 2, which will win last
last_winner = False
for pulled_number in keys:
    for indiv_bingo_card in bingo_card_deck:
        if indiv_bingo_card['Won']: 
            continue  # If the card is won we dont have to do anything else to it.
        
        if str(pulled_number) in indiv_bingo_card['Deck'].values:
            where_am_i = indiv_bingo_card['Deck'].isin([str(pulled_number)])
            coord = np.where(where_am_i)
            coordinates = [(x,y) for x, y in zip(coord[0], coord[1])]
            
            for coordinate in coordinates: # I dont think this is needed, but I am going to keep it just in case.
                indiv_bingo_card['Selected'].iloc[coordinate] = 1
            
            # Do any of the rows or columns add up to 5?
            # What about the two diagonals?
            winner = any([any(indiv_bingo_card['Selected'].apply(sum, axis = 0) == bingo_card_dimension), 
                          any(indiv_bingo_card['Selected'].apply(sum, axis = 1) == bingo_card_dimension),
                          np.diag(indiv_bingo_card['Selected']).sum() == bingo_card_dimension,
                          np.diag(indiv_bingo_card['Selected'].iloc[::-1]).sum() == bingo_card_dimension])
            
            if winner:
                indiv_bingo_card['Won'] = True
                if sum(list(map(lambda l: l['Won'], bingo_card_deck))) == len(bingo_card_deck):
                    print('We have our last winner!')
                    last_winner = True
                    last_winning_card = indiv_bingo_card
                
        if last_winner: break
    if last_winner: break

We have our last winner!


In [203]:
not_selectd_values = last_winning_card['Selected'].isin([str(0)])
coord = np.where(not_selectd_values)
coordinates = [(x,y) for x, y in zip(coord[0], coord[1])]
sum_of_not_selected_numbers = 0
for coordinate in coordinates:
    sum_of_not_selected_numbers += int(last_winning_card['Deck'].iloc[coordinate])
sum_of_not_selected_numbers * pulled_number

1830